# Imports

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from rich import print
import os

In [8]:
%%time
# https://afdc.energy.gov/fuels/electricity_locations.html#/analyze
file = '../data/external/alt_fuel_stations (Jan 5 2024).csv'
df = pd.read_csv(file)
df = df[(df['Fuel Type Code'] == 'ELEC') & (df['Country'] == 'US')]
df.info(memory_usage='deep')

<timed exec>:3: DtypeWarning: Columns (15,16,20,31,33,34,39,41,43,46,49,52,57,58,59,60,61,62,65,67,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
Index: 60634 entries, 87 to 80408
Data columns (total 74 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Fuel Type Code                           60634 non-null  object 
 1   Station Name                             60633 non-null  object 
 2   Street Address                           60634 non-null  object 
 3   Intersection Directions                  1940 non-null   object 
 4   City                                     60630 non-null  object 
 5   State                                    60621 non-null  object 
 6   ZIP                                      60634 non-null  object 
 7   Plus4                                    0 non-null      float64
 8   Station Phone                            60016 non-null  object 
 9   Status Code                              60634 non-null  object 
 10  Expected Date                            0 non-nul

In [30]:
# Are all of these open as of today?
df['Open Date'] = pd.to_datetime(df['Open Date'])
len(df[df['Open Date'].fillna(value=pd.Timestamp.today()) < pd.Timestamp.today()]) == len(df)

True

In [47]:
dc_plug_counts_by_network = df.dropna(subset='EV DC Fast Count').groupby('EV Network')['EV DC Fast Count'].sum().reset_index()
dc_plug_counts_by_network

,EV Network,EV DC Fast Count
0,7CHARGE,92.0
1,AMPUP,5.0
2,BP_PULSE,79.0
3,Blink Network,187.0
4,CHARGELAB,22.0
5,CHARGEUP,23.0
6,CIRCLE_K,100.0
7,ChargePoint Network,2396.0
8,EV Connect,818.0
9,EVCS,362.0


In [49]:
non_tesla_plugs = dc_counts_by_network[~dc_plug_counts_by_network['EV Network'].str.contains('Tesla')]
tesla_plugs = dc_counts_by_network[dc_plug_counts_by_network['EV Network'].str.contains('Tesla')]
tesla_plugs

,EV Network,EV DC Fast Count
29,Tesla,23679.0
30,Tesla Destination,16.0


In [50]:
print(f"{tesla_plugs['EV DC Fast Count'].sum()=:,}")
print(f"{non_tesla_plugs['EV DC Fast Count'].sum()=:,}")

tesla_plugs['EV DC Fast Count'].sum()=23,695.0

non_tesla_plugs['EV DC Fast Count'].sum()=14,310.0

In [51]:
# How much revenue across an arbitrary # of stations?
# Assume (non-Tesla) 4x plugs per station and $2,640 of monthly recurring revenue per station [https://blog.evbox.com/make-money-ev-charging-stations#:~:text=EV%20charging%20station%20revenue%20overview]
annual_revenue_per_dc_station = 2640  * 12
annual_revenue_per_dc_plug = annual_revenue_per_dc_station / 4
annual_revenue_per_dc_plug

7920.0

In [91]:
annual_revenue_estimate_per_dc_plug

39600.0

In [92]:
annual_revenue_estimate_per_level2_plug

9045.000000000002

In [90]:
annual_revenue_per_plug

39600.0

In [93]:
all_networks_revenue_annual

566676000.0

In [62]:
all_networks_dc_revenue_annual = annual_revenue_per_dc_plug * non_tesla_plugs['EV DC Fast Count'].sum()
print(f"${all_networks_dc_revenue_annual:,}")

$113,335,200.0

In [63]:
print(f"${all_networks_dc_revenue_annual*0.25:,}")

$28,333,800.0

In [64]:
28_333_800 * 15

425007000

In [60]:
# What will this number of lost revenue be in 2030 when we have more DCFC public plugs?
# https://www.mckinsey.com/features/mckinsey-center-for-future-mobility/our-insights/can-public-ev-fast-charging-stations-be-profitable-in-the-united-states
# Assume 4 plugs per station, similar proportion of non-Tesla-to-Tesla station counts
non_tesla_fraction = non_tesla_plugs['EV DC Fast Count'].sum() / dc_counts_by_network['EV DC Fast Count'].sum()
print(f"{non_tesla_fraction=}")
mckinsey_2030_public_dcfc_plug_multiplier = 1.5/0.1 # need multiplier since plug counts don't line up with McK estimate
print(f"{mckinsey_2030_public_dcfc_plug_multiplier=:,}")
plug_count_2030 = mckinsey_2030_public_dcfc_plug_multiplier * dc_counts_by_network['EV DC Fast Count'].sum()
print(f"{plug_count_2030=:,}")
print(f"Expected number of non-Tesla DCFC stations in US by 2030: {non_tesla_fraction * plug_count_2030 / 4:,}")
non_tesla_plugs_annual_revenue_2030 = non_tesla_fraction * plug_count_2030 * annual_revenue_per_dc_plug
print(f"Revenue for all non-Tesla DCFC plugs by 2030: ${non_tesla_plugs_annual_revenue_2030:,}")

non_tesla_fraction=0.3765294040257861

mckinsey_2030_public_dcfc_plug_multiplier=15.0

plug_count_2030=570,075.0

Expected number of non-Tesla DCFC stations in US by 2030: 53,662.50000000001

Revenue for all non-Tesla DCFC plugs by 2030: $1,700,028,000.0000002

In [61]:
print(f"25% lost revenue due to down plugs: ${non_tesla_plugs_annual_revenue_2030*0.25:,}")

25% lost revenue due to down plugs: $425,007,000.00000006

In [83]:
from evlens.data.markets import clean_adfc_charging_stations_data

df = clean_adfc_charging_stations_data(file, include_level2=False, revenue_loss_plug_fraction=0.25)

/Users/davemcrench/Documents/Projects/evlens/evlens/data/markets.py:34: DtypeWarning: Columns (15,16,20,31,33,34,39,41,43,46,49,52,57,58,59,60,61,62,65,67,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  print(f"${all_networks_dc_revenue_annual*0.25:,}")


Number of Tesla plugs: 23,695

Number of non-Tesla plugs: 14,310

2023 Non-Tesla Revenue: $566,676,000.0

2023 Non-Tesla Revenue Lost: $141,669,000.0

non_tesla_fraction=0.3765294040257861

plug_count_2030=570,075.0

Expected number of non-Tesla stations in US by 2030: 53,662.50000000001

2030 non-Tesla revenue by 2030: $8,500,140,000.000001

Lost 2030 non-Tesla revenue due to down plugs: $2,125,035,000.0000002

In [104]:
print(f"{45_000*14_310:,}")

643,950,000

In [102]:
from evlens.data.markets import clean_adfc_charging_stations_data

df = clean_adfc_charging_stations_data(file, include_level2=False, revenue_loss_plug_fraction=0.25)

/Users/davemcrench/Documents/Projects/evlens/evlens/data/markets.py:34: DtypeWarning: Columns (15,16,20,31,33,34,39,41,43,46,49,52,57,58,59,60,61,62,65,67,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Number of Tesla plugs: 23,695

Number of non-Tesla plugs: 14,310

2023 Non-Tesla Revenue: $566,676,000.0

2023 Non-Tesla Revenue Lost: $141,669,000.0

non_tesla_fraction=0.3765294040257861

plug_count_2030=570,075.0

Expected number of non-Tesla stations in US by 2030: 53,662.50000000001

2030 non-Tesla revenue by 2030: $8,500,140,000.0

Lost 2030 non-Tesla revenue due to down plugs: $2,125,035,000.0

In [105]:
# What happens to numbers if we include L2 chargers?
df_l2 = clean_adfc_charging_stations_data(file, include_level2=True, revenue_loss_plug_fraction=0.25)

/Users/davemcrench/Documents/Projects/evlens/evlens/data/markets.py:34: DtypeWarning: Columns (15,16,20,31,33,34,39,41,43,46,49,52,57,58,59,60,61,62,65,67,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Number of Tesla plugs: 40

Number of non-Tesla plugs: 5,122

2023 Non-Tesla Revenue: $112,541,175.0

2023 Non-Tesla Revenue Lost: $28,135,293.75

non_tesla_fraction=0.9922510654784967

plug_count_2030=77,430.0

Expected number of non-Tesla stations in US by 2030: 19,207.5

2030 non-Tesla revenue by 2030: $1,688,117,625.0

Lost 2030 non-Tesla revenue due to down plugs: $422,029,406.25

In [107]:
plug_counts_by_network

,EV Network,EV DC Fast Count,EV Level2 EVSE Num,Total
0,AMPUP,2.0,2.0,4
1,Blink Network,78.0,153.0,231
2,CHARGELAB,10.0,7.0,17
3,CHARGEUP,4.0,11.0,15
4,ChargePoint Network,3.0,8.0,11
5,EV Connect,145.0,178.0,323
6,EVCS,144.0,219.0,363
7,EVGATEWAY,15.0,67.0,82
8,Electrify America,346.0,113.0,459
9,FCN,142.0,98.0,240


In [108]:
include_level2 = True

if include_level2:
        columns_of_interest = ['EV DC Fast Count', 'EV Level2 EVSE Num']
else:
    columns_of_interest = ['EV DC Fast Count']
    
plug_counts_by_network = df.dropna(subset=columns_of_interest, how='all')\
    .groupby('EV Network')[columns_of_interest].sum().reset_index()
plug_counts_by_network['Total'] = \
    plug_counts_by_network[columns_of_interest].sum(axis=1).astype(int)
# dc_plug_counts_by_network.drop(columns=columns_of_interest, inplace=True)
    
non_tesla_plugs = plug_counts_by_network[
    ~plug_counts_by_network['EV Network'].str.contains('Tesla')
]
tesla_plugs = plug_counts_by_network[
    plug_counts_by_network['EV Network'].str.contains('Tesla')
]

print(f"Number of Tesla plugs: {tesla_plugs['Total'].sum():,}")
print(f"Number of non-Tesla plugs: {non_tesla_plugs['Total'].sum():,}")

# How much revenue across an arbitrary # of stations?
# Assume (non-Tesla) 4x plugs per station and $2,640 of monthly recurring 
# revenue per station 
# https://blog.evbox.com/make-money-ev-charging-stations#:~:text=EV%20charging%20station%20revenue%20overview
if include_level2:
    non_tesla_annual_revenue = \
        non_tesla_plugs['EV Level2 EVSE Num'].sum() * annual_revenue_estimate_per_level2_plug \
            + non_tesla_plugs['EV DC Fast Count'].sum() * annual_revenue_estimate_per_dc_plug
else:
    non_tesla_annual_revenue = \
        non_tesla_plugs['EV DC Fast Count'].sum() * annual_revenue_estimate_per_dc_plug
print(f"2023 Non-Tesla Revenue: ${non_tesla_annual_revenue:,}")
print(f"2023 Non-Tesla Revenue Lost: ${non_tesla_annual_revenue*revenue_loss_plug_fraction:,}")

# What will this number of lost revenue be in 2030 when we have more DCFC public plugs?
# https://www.mckinsey.com/features/mckinsey-center-for-future-mobility/our-insights/can-public-ev-fast-charging-stations-be-profitable-in-the-united-states
# Assume 4 plugs per station, similar proportion of non-Tesla-to-Tesla station counts
non_tesla_fraction = non_tesla_plugs['Total'].sum() / plug_counts_by_network['Total'].sum()
print(f"{non_tesla_fraction=}")

public_plug_multiplier_2030 = 1.5/0.1 # need multiplier since plug counts don't line up with McK estimate

plug_count_2030 = public_plug_multiplier_2030 * plug_counts_by_network['Total'].sum()
print(f"{plug_count_2030=:,}")
print(f"Expected number of non-Tesla stations in US by 2030: {non_tesla_fraction * plug_count_2030 / 4:,}")
non_tesla_plugs_annual_revenue_2030 = public_plug_multiplier_2030 * non_tesla_annual_revenue

# DCFC stats
print(f"2030 non-Tesla revenue by 2030: ${non_tesla_plugs_annual_revenue_2030:,}")
print(f"Lost 2030 non-Tesla revenue due to down plugs: ${non_tesla_plugs_annual_revenue_2030*revenue_loss_plug_fraction:,}")

Number of Tesla plugs: 36,707

Number of non-Tesla plugs: 123,942

2023 Non-Tesla Revenue: $1,558,297,440.0000002

2023 Non-Tesla Revenue Lost: $389,574,360.00000006

non_tesla_fraction=0.7715080703894827

plug_count_2030=2,409,735.0

Expected number of non-Tesla stations in US by 2030: 464,782.5

2030 non-Tesla revenue by 2030: $23,374,461,600.000004

Lost 2030 non-Tesla revenue due to down plugs: $5,843,615,400.000001

Something is wrong, including L2 shouldn't drop the lost revenue estimates...I think?